# Test perfect training accuracy on iris 

In [1]:
import math

from rerf.rerfClassifier import rerfClassifier

# Import scikit-learn dataset library
from sklearn import datasets

# Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.wilcoxon.html
from scipy.stats import mannwhitneyu

# testing sklearn random forest
from sklearn.ensemble import RandomForestClassifier

from tqdm import tqdm

In [2]:
# Load dataset
iris = datasets.load_iris()

In [3]:
def iris_pred_acc(cls, n_iter=10000):
    y_train_acc_list = []
    for i in tqdm(range(n_iter)):
        clf.fit(iris.data, iris.target)
        y_pred_train = clf.predict(iris.data)
        y_train_acc_list.append(metrics.accuracy_score(iris.target, y_pred_train))
        
    return y_train_acc_list 

In [4]:
def print_pred_summ(acc_list):
    print(sum([math.isclose(yt, 1) for yt in acc_list]))
    # print("avg acc", sum(y_train_acc_list)/len(y_train_acc_list))
    print(sorted(acc_list)[0:5])

In [5]:
clf = rerfClassifier(n_estimators=100, projection_matrix="RerF")
rerf_acc = iris_pred_acc(clf, 10000)
print("RerF")
print_pred_summ(rerf_acc)

clf = rerfClassifier(n_estimators=100, projection_matrix="Base")
rf_acc = iris_pred_acc(clf, 10000)
print("RF")
print_pred_summ(rf_acc)

clf = RandomForestClassifier(n_estimators=100)
sklearn_acc = iris_pred_acc(clf, 10000)
print("sklearn")
print_pred_summ(sklearn_acc)

  0%|          | 18/10000 [00:00<00:58, 171.45it/s]

RerF
9997
[0.9933333333333333, 0.9933333333333333, 0.9933333333333333, 1.0, 1.0]


  0%|          | 2/10000 [00:00<09:54, 16.81it/s]

RF
9966
[0.9933333333333333, 0.9933333333333333, 0.9933333333333333, 0.9933333333333333, 0.9933333333333333]


100%|██████████| 10000/10000 [09:06<00:00, 17.39it/s]

sklearn
9972
[0.9933333333333333, 0.9933333333333333, 0.9933333333333333, 0.9933333333333333, 0.9933333333333333]


In [9]:
def two_sided_mannwhitneyu(x,y):
    u, prob_one_sided = mannwhitneyu(x, y, use_continuity=False)
    prob = prob_one_sided*2

    return u, prob

In [10]:
_, prob_rerf_rf = two_sided_mannwhitneyu(rerf_acc, rf_acc)
_, prob_rerf_sklearn = two_sided_mannwhitneyu(rerf_acc, sklearn_acc)
_, prob_rf_sklearn = two_sided_mannwhitneyu(rf_acc, sklearn_acc)

print(f"RerF vs. RF:      p={prob_rerf_rf:0.15f}")
print(f"RerF vs. sklearn: p={prob_rerf_sklearn:0.15f}")
print(f"RF vs. sklearn:   p={prob_rf_sklearn:0.15f}")

RerF vs. RF:      p=0.000000337931825
RerF vs. sklearn: p=0.000007006066792
RF vs. sklearn:   p=0.445364679299948
